## **PARSING TINKOFF VACANCY**

In [1]:
############################################
###     Research Trending Vacancies      ###
###     Sber Dep. Research&Innovation    ### 
###   Ivanov Arseny, Sergey Bratchikov   ###
###       A. Efimov, D. Asonov           ###
############################################

In [2]:
import time
import requests
from bs4 import BeautifulSoup
import json
import re
import time
import faker
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from dateutil import parser

In [3]:
# fake = faker.Faker(locale='ru')

In [4]:
# tinkoff_headers = {
#     'User-Agent': fake.chrome(),
#     'accept-language': 'en-US,en;q=0.9',
#     'pragma': 'no-cache',
#     'Host': 'cfg.tinkoff.org',
#     'Origin': 'https://www.tinkoff.ru',
#     'content-type': 'application/json',
#     'referer': 'https://www.tinkoff.ru'
# }

In [5]:
API_URL = "https://cfg.tinkoff.ru/vacancies/public/api/platform/v1/getVacanciesList"
VACANCY_API_URL = "https://cfg.tinkoff.ru/vacancies/public/api/platform/v2/getVacancy?urlSlug="

In [6]:
clear_string = lambda x: re.sub(' +', ' ', re.sub('<.*?>', ' ', x).replace('\n', '\n ')).strip()

In [7]:
def build_job_infos(cat='it'):
    search_payload = {
        'category': cat,
        'filters': {},
        'limit': 200
    }
    result = requests.post(API_URL, data=search_payload)
    job_infos = []
    for category in result.json()['response']['items']:
        for job_info in category['items']:
            title = job_info['title']
            url = VACANCY_API_URL + job_info['id']
            job_infos.append((title, url))
    return job_infos

In [8]:
total_jobs = build_job_infos('it') + build_job_infos('back-office')
len(total_jobs)

161

In [9]:
job_dicts = []
for title, url in tqdm(total_jobs):
    result = requests.get(url).json()['response']

    job_dict = {
        'title': title,
        'internal_id' : result['id'],
        'url': url,
        'description': clear_string(result['tasks']),
        'responsibilities': clear_string(result['responsibilities']),
        'qualifications': clear_string(result['requirements']),
        'company': 'Tinkoff',
        'publish_date': parser.parse(result['specialtyInfo']['createdAt'])
    }

    job_dicts.append(job_dict)

    time.sleep(0.2)  # 200 ms

100%|██████████| 161/161 [00:42<00:00,  3.83it/s]


In [10]:
snapshot = pd.DataFrame(job_dicts)
print(len(snapshot))
snapshot.sample(5)

161


,title,internal_id,url,description,responsibilities,qualifications,company,publish_date
78,Продуктовый аналитик (Регуляторные процессы),0b84a4bc-ac29-4912-a676-6a34c3bfbe35,https://cfg.tinkoff.ru/vacancies/public/api/pl...,На развитие и мониторинг регуляторных процессо...,Исследовать процессы компании Разрабатывать мо...,"Системное мышление, развитая логика, умение до...",Tinkoff,2021-07-09 08:48:59.309000+00:00
94,Аналитик (Compliance-риски),58278060-89f9-4d15-927d-8f21abb9caf1,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Ищем опытного аналитика для решения широкого к...,Построение и валидация скоринговых и рисковых ...,Оконченное высшее техническое образование (пре...,Tinkoff,2018-03-22 07:48:44.260000+00:00
21,Golang-разработчик в DCO,da3fd5ed-4166-43b0-ae6d-2485c3c96b6b,https://cfg.tinkoff.ru/vacancies/public/api/pl...,В Тинькофф мы стараемся сделать наши продукты ...,Разработка и поддержка бекэнда в условиях высо...,Требования Знание Go и C++ Знание сетевого сте...,Tinkoff,2018-03-27 09:04:23.643000+00:00
48,Системный аналитик в Инвестиции,219c9aa5-8760-424f-8d38-c2a371088146,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Tinkoff.ru всего за год создал собственного ун...,"Подготовка спецификаций (сбор требований, анал...",Опыт работы системным аналитиком от 3 лет Навы...,Tinkoff,2021-07-23 13:56:43.353000+00:00
158,HR Бизнес партнер,bc5f0540-28f5-4d0a-9e1f-714f75974b2a,https://cfg.tinkoff.ru/vacancies/public/api/pl...,Ищем HR Бизнес партнера в нашу команду,Отслеживание и работа над мотивацией и вовлече...,Опыт работы в качестве HR бизнес-партнера / HR...,Tinkoff,2018-03-22 07:48:04.899000+00:00


In [11]:
snapshot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161 entries, 0 to 160
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype                  
---  ------            --------------  -----                  
 0   title             161 non-null    object                 
 1   internal_id       161 non-null    object                 
 2   url               161 non-null    object                 
 3   description       161 non-null    object                 
 4   responsibilities  161 non-null    object                 
 5   qualifications    161 non-null    object                 
 6   company           161 non-null    object                 
 7   publish_date      161 non-null    datetime64[ns, tzutc()]
dtypes: datetime64[ns, tzutc()](1), object(7)
memory usage: 10.2+ KB


In [12]:
current_date = datetime.now().strftime('%d-%m-%Y')
current_date

'03-07-2022'

In [13]:
snapshot.to_csv(f'../data/tinkoff/{current_date}.csv')

In [135]:
requests.get(total_jobs[10][1]).json()['response']

{'id': '789e6068-504b-4278-baab-477a156fd814',
 'status': 'published',
 'title': 'PL/SQL разработчик',
 'specialty': 'Backend',
 'specialtyInfo': {'createdAt': '2018-03-27T09:04:23.643Z',
  'deletedAt': None,
  'id': '4c5c5e87-d961-4c74-a385-d0b25e36cdc5',
  'name': 'Backend',
  'order': 1,
  'urlSlug': 'back-end-razrabotka'},
 'city': 'Любой город',
 'experience': 'Middle',
 'tasks': '<p>Тинькофф — это огромная экосистема. Нашими клиентами уже стали 13 млн человек по всей России. Мы активно развиваемся, появляются новые продукты и сервисы, за которыми стоят разработчики и инженеры. Наши сервисы должны соответствовать высочайшим стандартам надежности и быстродействия. Поэтому сейчас мы ищем разработчика PL/SQL для усиления команд.</p>',
 'requirements': '<ul><li>Отличное знаешь PL/SQL (Oracle 10g и выше)</li><li>Понимаешь архитектуру oracle, основные структуры данных (индексы, таблицы), алгоритмы доступа к данным (index range/full/skip scan) и их соединения</li><li>Знаешь SQL и имеешь 